In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

from plotnine import *

# Example: Finding Jobs by County over Years

In this section, we will combine everything we have learned in this book so far to create a workflow for getting the number of jobs by county over many years. Here are the steps we will take:

- Get WAC data and crosswalk from the internet.
- Store the data within a database.
- Use SQL to join the WAC to the crosswalk to get the number of jobs by county.
- Create a Data Frame which we can use to create a visualization.
- Graph the jobs over years by county.

Within this whole process, we will use functions and loops to make it easier to do this over many years.

## Creating a New Database

We first start by creating a new database to store our data. Creating a connection to a new database name that doesn't exist will do this for us.

In [8]:
conn = sqlite3.connect("md_lodes.db")

Next, we define a function that will help us get the WAC data that we want. Try to read through the function and determine what it does. Read back to the previous Python material if you aren't sure about some parts.

In [3]:
def store_wac_md(year, con):
    '''
    Gets WAC data for Maryland for a given year and stores in a database in the connection
    
    Arguments:
    year: Year to get data for
    con: Connection to the database
    
    Returns:
    None
    '''
    
    # Bring in data from internet
    base_url = 'https://lehd.ces.census.gov/data/lodes/LODES7/'
    file_specs = 'md/wac/md_wac_S000_JT00_{yr}.csv.gz'.format(yr = year)
    file_name = base_url + file_specs
    df_md = pd.read_csv(file_name,compression='gzip')
    
    # Store in database
    df_md.columns = df_md.columns.str.lower()
    df_md.to_sql('md_wac_' + str(year), conn, if_exists='replace')

We then use a loop to store the data within the database we created.

In [4]:
for i in range(2007,2017):
    store_wac_md(i, conn)

We just need to run the process for bringing in the crosswalk data once. We'll store that in the database too.

In [7]:
data_url = 'https://lehd.ces.census.gov/data/lodes/LODES7/md/md_xwalk.csv.gz'
md_xwalk = pd.read_csv(data_url,compression='gzip')
md_xwalk.columns = md_xwalk.columns.str.lower()
md_xwalk.to_sql('md_xwalk', conn, if_exists='replace')

145247

We're going to need another function to automate doing the join, then bring that into Python as a Data Frame.

In [ ]:
def jobs_by_county(year, conn):
    '''
    Joins the WAC to the xwalk for a given year and calculates jobs by county. 
    
    Arguments:
    year: Year to get data for
    con: Connection to the database
    
    Returns:
    A Data Frame with three columns, county, jobs, and year
    '''    
    
    base_query = '''
    SELECT ctyname as county, count(*) as jobs, {year} as year
    FROM md_wac_{year} wac
    JOIN md_xwalk xwalk
    ON wac.w_geocode = xwalk.tabblk2010
    GROUP BY ctyname
    '''
    
    return pd.read_sql(base_query.format(year = year), conn)

In [ ]:
jobs_by_county(2015,conn)

We'll use a loop to get the data we want stacked on top of each other so that we have a long dataset with three columns: county, jobs, and year.

In [6]:
md_jobs_by_county = pd.DataFrame()
for i in range(2007,2017):
    md_jobs_by_county = pd.concat([md_jobs_by_county, jobs_by_county(i,conn)], 
                                  ignore_index= True)
md_jobs_by_county.head()

NameError: name 'jobs_by_county' is not defined

Finally, we can plot the jobs over years by county. Here, we use ggplot to do the plotting, but you can use other methods as well.

In [ ]:
(ggplot(md_jobs_by_county)
 + aes(x = 'year', y = 'jobs', color = 'county') 
 + geom_line())

## <span style="color:red">Checkpoint: Python and SQL Workflow</span>

Make sure you understand how all of the code above comes together. Try adjusting the code above so that the graph only has a few counties so that it is easier to read. Try doing the same for a different state. 